### Post Import Fixes
> Post fixes that need to be applied after importing litesearch package.

We need to apply some post fix for mac to load the dylib properly for usearch sqlite path to be loaded correctly.


In [13]:
#| default_exp postfix

In [14]:
#| export
import os, subprocess, platform

In [15]:
#| export
def usearch_fix():
    """Apply usearch macOS fix if on macOS."""
    print('Applying usearch macOS fix if required...')
    try:
        from usearch import sqlite_path
        dylib_path = sqlite_path()+'.dylib'
        print('usearch dylib path: ', dylib_path)
        if platform.system() != "Darwin":
            print('Not on macOS, skipping usearch fix.')
            return
        cmd = ['install_name_tool', '-add_rpath', '/usr/lib', dylib_path]
        r = subprocess.run(cmd, capture_output=True, text=True, check=True)
        if r.returncode == 0:  print(f'✓ Applied usearch fix: Added /usr/lib rpath to {dylib_path}')
        else: print(f'✗ Failed to apply fix: {r.stderr}')
    except ImportError as ie:
        print('Warning: usearch not installed or import failed. you might need to install libsqlite3-dev. '
              'For macs do `brew install libsqlite3-dev`. For linux `apt install libsqlite3-dev`. ', ie)
    except subprocess.CalledProcessError as e:
        out = subprocess.run(['otool', '-l', dylib_path], capture_output=True, text=True).stdout
        if '/usr/lib' in out:
            print('rpath already present. Skip.')
            return
        print(f'✗ install_name_tool failed: {e}')
        print(f'Command output: {e.output}')
        print(f'Command stderr: {e.stderr}')
    except Exception as e: print(f'Unexpected error during fix: {e}')

In [16]:
#| hide
usearch_fix()

Applying usearch macOS fix if required...
usearch dylib path:  /Users/71293/code/litesearch/.venv/bin/usearch_binaries/usearch_sqlite.dylib
rpath already present. Skip.
